# Task 1: Mark all launch sites on a map

In [1]:
import folium
import pandas as pd

# Load the dataset with launch site coordinates
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')

# Extract relevant columns
launch_sites_df = spacex_df.groupby('Launch Site').first().reset_index()[['Launch Site', 'Lat', 'Long']]

# Start location for the map (NASA Johnson Space Center)
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add each launch site as a circle and marker on the map
for index, site in launch_sites_df.iterrows():
    folium.Circle(
        radius=1000,
        location=[site['Lat'], site['Long']],
        popup=site['Launch Site'],
        color='crimson',
        fill=True,
    ).add_to(site_map)

    folium.Marker(
        location=[site['Lat'], site['Long']],
        popup=site['Launch Site'],
        icon=folium.DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site['Launch Site'],
        )
    ).add_to(site_map)

# Display the map
site_map


# Task 2: Mark the success/failed launches for each site on the map1

In [2]:
from folium.plugins import MarkerCluster

# Initialize MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Loop through each launch record
for index, record in spacex_df.iterrows():
    # Determine the marker color based on the launch outcome
    if record['class'] == 1:
        marker_color = 'green'
    else:
        marker_color = 'red'

    # Create a Marker object
    folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=marker_color),
        popup=f"Launch Site: {record['Launch Site']}, Outcome: {'Success' if record['class'] == 1 else 'Failed'}",
    ).add_to(marker_cluster)

# Display the updated map
site_map


# Task 3: Calculate the distances between a launch site to its proximities

In [3]:
from folium.plugins import MousePosition
from math import radians, sin, cos, sqrt, atan2

# Add MousePosition to get coordinate for a mouse over
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# Function to calculate distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # approximate radius of Earth in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Example: Calculate distance to a coastline point (replace with actual coordinates)
coastline_lat = 28.56367
coastline_lon = -80.57163
launch_site_lat = launch_sites_df.loc[0, 'Lat']
launch_site_lon = launch_sites_df.loc[0, 'Long']

distance_to_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a marker for coastline distance
folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=folium.DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_to_coastline),
    )
).add_to(site_map)

# Draw a polyline between launch site and coastline
lines = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]], weight=1)
site_map.add_child(lines)

# Display the updated map
site_map
